# Set up environment

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import
from __future__ import with_statement
from __future__ import division
from __future__ import nested_scopes
from __future__ import generators
from __future__ import unicode_literals
from __future__ import print_function

# Load scipy/numpy/matplotlib
from   scipy.linalg import expm
import matplotlib.pyplot as plt
from   pylab import *
import numpy as np
import scipy.io
from warnings import warn

# Configure figure resolution
plt.rcParams['figure.figsize'] = (12.0, 6.0)
plt.rcParams['savefig.dpi'   ] = 100

from izh       import * # Routines for sampling Izhikevich neurons
from plot      import * # Misc. plotting routines
from glm       import * # GLM fitting
from arppglm   import * # Sampling and integration
from utilities import * # Other utilities
from arguments import * # Argument verification

'''
import os
dtype='float64'
flags = 'mode=FAST_RUN,device=gpu,floatX=%s'%dtype
if dtype!='float64':
    flags += ',warn_float64=warn'
os.environ["THEANO_FLAGS"] = flags
import theano
import theano.tensor as T
'''

import os
dtype='float32'
os.environ['MKL_THREADING_LAYER']='GNU'
flags = 'mode=FAST_COMPILE,device=cuda0,'#,floatX=%s'%dtype
if dtype!='float64':
     flags += ',warn_float64=warn'
os.environ["THEANO_FLAGS"] = flags

import theano
import theano.tensor as T

from theano_arppglm import *

print('Workspace Initialized')

# Load saved GLM features

In [ ]:
#filename = 'saved_training_model.mat'
filename = 'saved_training_model_badburster.mat'

saved_training_model = scipy.io.loadmat(filename)
K  = np.array(saved_training_model['K'],dtype=dtype)
B  = np.array(saved_training_model['B'],dtype=dtype)
By = np.array(saved_training_model['By'],dtype=dtype)
Bh = np.array(saved_training_model['Bh'],dtype=dtype)
A  = np.array(saved_training_model['A'],dtype=dtype)
C  = np.array(saved_training_model['C'],dtype=dtype)
Y  = np.array(saved_training_model['Y'],dtype=dtype)
dt = np.array(saved_training_model['dt'],dtype=dtype)

Bh_train = saved_training_model['Bh_train']
By_train = saved_training_model['By_train']
X_train  = concatenate([By_train,Bh_train],axis=1)
Y_train  = asvector(saved_training_model['Y_train'])

Bh_test  = saved_training_model['Bh_test']
By_test  = saved_training_model['By_test']
X_test   = concatenate([By_test,Bh_test],axis=1)
Y_test   = asvector(saved_training_model['Y_test'])
 
K  = int(scalar(K))
N  = prod(Y.shape)

N = len(X_train)
STARTPLOT=0
NPLOT=N

print('Saved GLM features loaded')
print(N)

#STARTSHOW = 14000
#STOPSHOW = 16000
STARTSHOW = 0
STOPSHOW = N

### GLM helpers

In [ ]:
def lograte(Bh,By,p):
    '''
    Log-intensity of point process model on this dataset
    Predicted using the standard GLM way
    '''
    m       = array(p).ravel()[0]
    beta    = ascolumn(p[1:K+1])
    beta_st = ascolumn(p[1+K:])
    lograte = m + Bh.dot(beta_st) + By.dot(beta)
    return lograte

def logmean(Bh,M1,p):
    '''
    Projected history process
    Predicted using history-process means
    '''
    m       = array(p).ravel()[0]
    beta    = ascolumn(p[1:K+1])
    beta_st = ascolumn(p[1+K:])
    M1      = np.squeeze(M1)
    return (beta.T.dot(M1.T))[0] + (m + Bh.dot(beta_st))[:,0]

def get_stim(Bh,p):
    m        = array(p).ravel()[0]
    beta     = ascolumn(p[1:K+1])
    beta_st  = ascolumn(p[1+K:])
    stim     = (m + Bh.dot(beta_st))[:,0]
    return stim

def filter_GLM_np(Bh,p):
    m        = array(p).ravel()[0]
    beta     = ascolumn(p[1:K+1])
    beta_st  = ascolumn(p[1+K:])
    stim     = get_stim(Bh,p)
    allM1_np = np.zeros((N,K))
    M1       = np.zeros((K,1))
    for i in range(N):
        R   = scalar(sexp(p0[1:K+1].dot(M1)+m+stim[i]))
        M1 += A.dot(M1)*dt + C.dot(R)
        allM1_np[i] = M1[:,0]
    return allM1_np

def addspikes_(Y_=None):
    if Y_ is None or Y_ is True:
        Y_ = Y
    for t in find(Y_>0):
        axvline(t,color=OCHRE,lw=0.4)
    
def niceaxis(plotspikes=True):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore",message='No labelled objects found')
        legend()
    simpleraxis()
    xlim(STARTSHOW,STOPSHOW)
    if plotspikes is True or not plotspikes is None:
        addspikes_(plotspikes)

print('GLM helpers done')

In [ ]:
# Re-fit GLM
m,bhat  = fitGLM(X_train,asvector(Y_train))

# Re-pack model parameters
p0      = np.zeros((1+len(bhat)))
p0[0 ]  = m
p0[1:]  = bhat

allM1_np = filter_GLM_np(Bh_train,p0)
subplot(311)
plot(lograte(Bh_train,By_train,p0),lw=0.4,label='conditional intensity')
plot(logmean(Bh_train,allM1_np,p0),lw=0.4,label='mean-field',color=RUST)
niceaxis()
ylim(min(lograte(Bh_train,By_train,p0)),5)

# Filtering 

### Parameters at which to filter

In [ ]:
oversample = 10   # Integration resolution
maxrate    = 10.0 # Largest allowed rate
maxvcorr   = 10.0 # Largest allowed variance correction
dt         = 1.0  # Data time resolution
reg_cov    = 1e-5
reg_rate   = 1e-5

p = p0.copy()
#p[1:K+1] *= 0.775
stim_np = get_stim(Bh_train,p)
beta_np = ascolumn(p[1:K+1])
print('Filtering using p=',v2str(p))

# Helper function to compute negative expected log-likelihood
def post_hoc_nll(LR,LV):
    R0 = sexp(LR)
    R1 = R0*(1+0.5*LV)
    ELL  = np.mean(Y*LR - R1)
    return -ELL

### Build Theano routines

For integrating moments (not conditioned on data), filtering (conditioned on data), and filtering using surrogate likelihoods (Gaussian approximations).

In [ ]:
from theano_arppglm import *

GLM_log_intensity, GLMNLL_f, GLMNLL_g, GLMNLL_h = build_ML_GLM_likelihood_theano()

In [ ]:
integrate_moments_theano, EMNLL_filt, EMNLL_grad = build_integrate_moments_theano(N,A,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler")

In [ ]:
filter_moments_theano, NLL_filt, NLL_grad = build_filter_moments_theano(N,A,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = True)

In [ ]:
filter_surrogate_theano, SNLL_filt, SNLL_grad = build_filter_moments_theano(N,A,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = False,
    use_surrogates    = True)

print('Theano functions bulit')

## Integrate Without measurements.

We will use the integrated moments (without measurements) as a prior distribution for our K-step prediction with measurement updates.

Update: we now use the actual filtered states, to verify that shallow filtering is equivalent to deep filtering if provided appropriate initial conditions

In [ ]:
p = p0.copy()
#p[1:]*=3

m        = array(p).ravel()[0]
beta     = ascolumn(p[1:K+1])
beta_st  = ascolumn(p[1+K:])
stim     = (m + Bh_train.dot(beta_st))[:,0]
stim_np  = stim
beta_np  = ascolumn(p[1:K+1])

print('Filtering using p=',v2str(p))

tic()
allLRni,allLVni,allM1ni,allM2ni = integrate_moments(stim_np,A,beta_np,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler")
toc()
subplot(411)
stderrplot(allLRni,allLVni,color=BLACK,lw=0.5)
niceaxis()
xlim(STARTSHOW,STOPSHOW)
title('Integrating, numpy')

tic()
allLRti,allLVti,allM1ti,allM2ti = integrate_moments_theano(Bh_train,p)
toc()
subplot(412)
stderrplot(allLRti,allLVti,color=BLACK,lw=0.5)
niceaxis()
xlim(STARTSHOW,STOPSHOW)
title('Integrating, theano')

tight_layout()

# Deep filtering in Numpy

In [ ]:
tic()
allLRn,allLVn,allM1n,allM2n,nlln,mrn,vrn = filter_moments(stim,Y_train,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = True)
toc()
subplot(411)
stderrplot(allLRn,allLVn,color=BLACK,lw=0.5)
niceaxis()
xlim(STARTSHOW,STOPSHOW)
title('Filtering, numpy')
print('nll, numpy',nlln)

tic()
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh_train,Y_train,p)
toc()
subplot(412)
stderrplot(allLRt,allLVt,color=BLACK,lw=0.5)
niceaxis()
xlim(STARTSHOW,STOPSHOW)
title('Filtering, theano')
print('nll, theano',nllt)

subplot(413)
plot(allLRn,color=BLACK,label='log-λ numpy')
plot(allLRt,':',color=RUST,label='log-λ theano')
niceaxis()
xlim(STARTSHOW,STOPSHOW)

priorLR,priorLV,priorM1,priorM2 = allLRt,allLVt,allM1t,allM2t

tight_layout()

# First: let us see if we can get "parallel" moment integration (no measurements). 

## Start with naive implementation for reference

Demonstrate shallow depth-5 filtering. Even starting from a prior with no inforamation about the filtered state, these results can be relatively accurate. This could lead to parallel filtering routines to accelerate inference. 

In [ ]:
from arppglm import filter_moments

In [ ]:
D  = 5
ND = N-D

# Precompute constants
maxlogr   = np.log(maxrate)
maxratemc = maxvcorr*maxrate
dtfine    = dt/oversample
Cb        = C.dot(beta.T)
CC        = C.dot(C.T)
Adt       = A*dtfine

themeasurement = 'moment'
int_method = 'euler'
method = 'second_order'

# Get measurement update function
measurement = get_measurement(themeasurement)
# Buid moment integrator functions
mean_update, cov_update = get_moment_integrator(int_method,Adt)
# Get update function (computes expected rate from moments)
update = get_update_function(method,Cb,Adt,maxvcorr)

allLR = np.zeros(N)
allLV = np.zeros(N)

tic()
for i in range(N):
    b = i+1
    a = i-D+1
    c = a-1
    ini = (priorM1[c],priorM2[c]) if c>=0 else None
    a = max(0,a)
    c = max(0,a-1)
    lr,lv,_,_,_,_,_ = filter_moments(stim[a:b],Y_train[a:b],A,beta,C,p[0],
        dt          = dt,
        oversample  = oversample,
        maxrate     = maxrate,
        maxvcorr    = maxvcorr,
        method      = "second_order",
        int_method  = "euler",
        measurement = "moment",
        reg_cov     = reg_cov,
        reg_rate    = reg_rate,
        return_surrogates = True,
        initial_conditions = ini)
    allLR[i] = lr[-1]
    allLV[i] = lv[-1]
toc()

assert(all(isfinite(allLR)))
assert(all(isfinite(allLV)))

allLRref,allLVref = allLR,allLV

subplot(311)
stderrplot(allLRn,allLVn,color=BLACK,lw=0.5)
stderrplot(allLR,allLV,color=RUST,lw=0.5,filled=0)
niceaxis()
ylim(max(ylim()[0],-100),5)

## Compartamentalize in function

Note that parallel shallow filtering is still slower than running the full forward pass, as we must perform O(D*N) work as opposed to O(N). However, this will admits a depth-D algorithm in theano which may give us some improvement

In [ ]:
tic()
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh_train,Y_train,p)
toc()

priorLR,priorLV,priorM1,priorM2 = allLRt,allLVt,allM1t,allM2t
defaultM1 = np.zeros((K,1))
defaultM2 = np.eye(K)*1e-6

iniM1 = np.zeros((N,K,1))
iniM2 = np.zeros((N,K,K))
iniM1[:D-1]=defaultM1
iniM2[:D-1]=defaultM2
iniM1[D-1:]=priorM1[:-D+1]
iniM2[D-1:]=priorM2[:-D+1]

In [ ]:
tic()
allLRn,allLVn,allM1n,allM2n,nlln,mrn,vrn = filter_moments(stim,Y_train,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = True)
toc()

from dstep import filter_moments_dstep

tic()
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh_train,Y_train,p)
toc()

priorLR,priorLV,priorM1,priorM2 = allLRt,allLVt,allM1t,allM2t
defaultM1 = np.zeros((K,1))
defaultM2 = np.eye(K)*1e-6

iniM1 = np.zeros((N,K,1))
iniM2 = np.zeros((N,K,K))
iniM1[:D]=defaultM1
iniM2[:D]=defaultM2
iniM1[D:]=priorM1[:-D]
iniM2[D:]=priorM2[:-D]


tic()
allLRnd,allLVnd,allM1nd,allM2nd,nllnd = filter_moments_dstep(D,stim,Y_train,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    prior       = (iniM1,iniM2))
toc()

subplot(311)
stderrplot(allLRref,allLVref,color=BLACK,lw=0.5)
stderrplot(allLRnd,allLVnd,color=RUST,lw=0.5,filled=0)
niceaxis()
ylim(max(ylim()[0],-100),5)

# compare likelihoods
# parallel shallow likelihood as close to filtered likelihood
# as it is to the theano implementation
# meaning that a shallow filter is as accurate as a deep filter
# up to numerical precision errors
print(nllt,nlln,nllnd)

# Theano implementation

Separate functions are great for debugging, but let's clean things up a bit!

In [ ]:
tic()
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh_train,Y_train,p)
toc()

priorLR,priorLV,priorM1,priorM2 = allLRt,allLVt,allM1t,allM2t
defaultM1 = np.zeros((K,1))
defaultM2 = np.eye(K)*1e-6

iniM1 = np.zeros((N,K,1))
iniM2 = np.zeros((N,K,K))
iniM1[:D-1]=defaultM1
iniM2[:D-1]=defaultM2
iniM1[D-1:]=priorM1[:-D+1]
iniM2[D-1:]=priorM2[:-D+1]

In [ ]:
allRC = np.zeros((N,K,K))
for i in range(N):
    allRC[i] = np.eye(K)

In [ ]:
S = stim
Y = Y_train

TAdt  = Tcon(Adt)
Tbeta = Tcon(beta)
Tb    = Tcon(p.ravel()[1:K+1])
TC    = Tcon(C ).dimshuffle('x',0,1)
TCb   = Tcon(Cb).dimshuffle('x',0,1)
TCC   = Tcon(CC).dimshuffle('x',0,1)
TRC   = Tcon(allRC)

TM1 = T.tensor3("TM1",dtype=dtype)
TM2 = T.tensor3("TM2",dtype=dtype)

mxl = Tcon(maxlogr)
mxr = Tcon(maxrate)
dtf = Tcon(dtfine)
xvc = Tcon(maxvcorr)
rr  = Tcon(reg_rate)
mm  = Tcon(m)

def integrate_moments_parallel_theano_source(M1,M2,S):
    LOGV = M2.dot(Tb).dot(Tb) # N
    LOGM = M1[:,:,0].dot(Tb)  # N
    LOGX = T.minimum(mxl,LOGM+S) # N
    R0   = T.minimum(mxr,Tsexp(LOGX))*dtf # N 
    RM   = R0 * T.minimum(1.0+0.5*LOGV,xvc) # N
    J    = TCb*R0[:,None,None]+TAdt[None,:,:]
    JM2  = T.batched_dot(J,M2)
    M2  += JM2 + JM2.transpose(0,2,1)    + TCC*RM[:,None,None]
    M1  += TAdt.dot(M1).transpose(1,0,2) + TC *RM[:,None,None]
    return M1,M2

Tintr = Tcon(np.linspace(-4,4,25))
def measurement_update_parallel_theano_source(M1,M2,S_,Y_):
    LV = M2.dot(Tb).dot(Tb) # N
    LM = Tcast(M1[:,:,0].dot(Tb)) # N
    LT = Tsinv(LV)
    TQ = LT + rr
    VQ = Tsinv(TQ)
    MQ = (LM*LT+mm*rr)*VQ
    X_ = Tintr[None,:]*T.sqrt(VQ)[:,None]+MQ[:,None]
    R0 = X_ + S_[:,None]+Tslog(dt)
    L  = Y_[:,None]*R0-Tsexp(R0)
    L  = L - T.max(L,axis=1)[:,None]
    L += -0.5*((Tintr**2.0)[None,:]+Tslog(VQ)[:,None])
    PR = T.maximum(Tcast(1e-7),Tsexp(L))
    NR = Tsinv(T.sum(PR,axis=1))
    MP = T.sum(X_*PR,axis=1)*NR
    VP = T.sum((X_-MP[:,None])**2.0*PR,axis=1)*NR
    TP = Tsinv(VP)
    VR = Tsinv(TP-LT)
    MR = (MP*TP-LM*LT)*VR
    # Multivariate conditional update
    M2B   = M2.dot(Tbeta)
    KG    = Tsdiv(M2B,(VR+LV)[:,None,None])
    M2   -= T.batched_dot(KG,M2B.transpose(0,2,1))
    M1   += KG*(MR-LM)[:,None,None]
    LR    = Tmn(mxl,M1[:,:,0].dot(Tb)+S_)
    LOGPYX= Y_*LR-Tsexp(LR)
    LL    = LOGPYX - 0.5*(Tslog(LV/VP) + (MP-LM)**2.0/LV)
    return M1,M2,-T.mean(LL)

def filter_moments_parallel_theano_source(di,M1,M2,S_,Y_):
    if reg_cov>0:
        M2 = 0.5*(M2 + M2.transpose(0,2,1)) + TRC
    offsets = T.maximum(0.0,T.arange(N)+di)
    offsets = T.cast(offsets,'int32')
    S_ = S_[offsets]
    Y_ = Y_[offsets]
    for k in range(oversample):
        M1,M2 = integrate_moments_parallel_theano_source(M1,M2,S_)
    M1,M2,NLL = measurement_update_parallel_theano_source(M1,M2,S_,Y_)
    return M1,M2,NLL

Tdi = T.scalar("Tdi",dtype=dtype)
TS_ = T.vector("TS_",dtype=dtype)
TY_ = T.vector("TY_",dtype=dtype)

filter_moments_parallel_theano = Tfun(
    inp = [Tdi,TM1,TM2,TS_,TY_],
    out = filter_moments_parallel_theano_source(Tdi,TM1,TM2,TS_,TY_))

# Depth D Loop
[_M1,_M2,_NLL], up = theano.scan(filter_moments_parallel_theano_source,
                                sequences     = [Tcon(arange(1-D,1))],
                                outputs_info  = [Tcon(iniM1),Tcon(iniM2),None],
                                non_sequences = [TS_,TY_],
                                n_steps       = D,
                                name          = 'scan_moments_parallel_theano')
#
M1,M2 = _M1[-1],_M2[-1]
ALLLV = M2.dot(Tb).dot(Tb) # N
ALLLR = T.minimum(maxlogr,M1[:,:,0].dot(Tb)+TS_) # N
scan_moments_parallel_theano = Tfun(\
    inp = [TS_,TY_],
    out = [ALLLR,ALLLV,M1,M2,_NLL[-1]],
    upd = up)

print('Theano shallow filter defined')

In [ ]:
tic()
#allLR,allLV,M1,M2,NLL = scan_moments_parallel_theano(Bh_train,Y,p)
allLR,allLV,M1,M2,NLL = scan_moments_parallel_theano(S,Y)#,p)
toc()

subplot(311)
stderrplot(allLRref,allLVref,color=BLACK,lw=0.5)
stderrplot(allLR,allLV,color=RUST,lw=0.5,filled=0)
niceaxis()

# compare likelihoods
# parallel shallow likelihood as close to filtered likelihood
# as it is to the theano implementation
# meaning that a shallow filter is as accurate as a deep filter
# up to numerical precision errors
print(nllt,nlln,NLL)
ylim(max(ylim()[0],-100),5) 

# Now we need to add support for surrogate measurements

## Redefine measurement to use surrogates

In [ ]:

MR_ = T.vector("MR_",dtype=dtype)
VR_ = T.vector("VR_",dtype=dtype)
TS_ = T.vector("TS_",dtype=dtype)

Tintr = Tcon(np.linspace(-4,4,25))
def measurement_update_parallel_theano_surrogate_source(M1,M2,S_,Y_,MR,VR):
    LV  = M2.dot(Tb).dot(Tb) # N
    LM  = Tcast(M1[:,:,0].dot(Tb)) # N
    LT  = Tsinv(LV)
    # Multivariate conditional update
    M2B = M2.dot(Tbeta)
    KG  = Tsdiv(M2B,(VR+LV)[:,None,None])
    M2 -= T.batched_dot(KG,M2B.transpose(0,2,1))
    M1 += KG*(MR-LM)[:,None,None]
    # Compute univariate update for likelihood
    TR  = Tsinv(VR)
    TP  = LT + TR
    VP  = Tsinv(TP)
    MP  = (LT*LM+TR*MR)*VP
    # Compute likelihood
    LR  = Tmn(mxl,M1[:,:,0].dot(Tb)+S_)
    LYX = Y_*LR-Tsexp(LR)
    LL  = LYX - 0.5*(Tslog(LV/VP) + (MP-LM)**2.0/LV)
    return M1,M2,-T.mean(LL)

def filter_moments_parallel_theano_surrogate_source(di,M1,M2,S_,Y_,MR,VR):
    if reg_cov>0:
        M2 = 0.5*(M2 + M2.transpose(0,2,1)) + TRC
    offsets = T.maximum(0.0,T.arange(N)+di)
    offsets = T.cast(offsets,'int32')
    S_ = S_[offsets]
    Y_ = Y_[offsets]
    MR = MR[offsets]
    VR = VR[offsets]
    for k in range(oversample):
        M1,M2 = integrate_moments_parallel_theano_source(M1,M2,S_)
    M1,M2,NLL = measurement_update_parallel_theano_surrogate_source(M1,M2,S_,Y_,MR,VR)
    return M1,M2,NLL

# Depth D Loop
[_M1,_M2,_NLL], up = theano.scan(filter_moments_parallel_theano_surrogate_source,
                                sequences     = [Tcon(arange(1-D,1))],
                                outputs_info  = [Tcon(iniM1),Tcon(iniM2),None],
                                non_sequences = [TS_,TY_,MR_,VR_],
                                n_steps       = D,
                                name          = 'scan_moments_parallel_theano')
#
M1,M2 = _M1[-1],_M2[-1]
ALLLV = M2.dot(Tb).dot(Tb) # N
ALLLR = T.minimum(maxlogr,M1[:,:,0].dot(Tb)+TS_) # N
scan_moments_parallel_theano_surrogate = Tfun(\
    inp = [TS_,TY_,MR_,VR_],
    out = [ALLLR,ALLLV,M1,M2,_NLL[-1]],
    upd = up)

print('Theano shallow filter using surrogate measurements defined')

## Test theano implementation with surrogate measurements

In [ ]:
# Get surrogate measurements
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh_train,Y_train,p)

tic()
allLR,allLV,M1,M2,NLL = scan_moments_parallel_theano_surrogate(S,Y,mrt,vrt)
toc()

subplot(311)
stderrplot(allLRref,allLVref,color=BLACK,lw=0.5)
stderrplot(allLR,allLV,color=RUST,lw=0.5,filled=0)
niceaxis()

# compare likelihoods
# parallel shallow likelihood as close to filtered likelihood
# as it is to the theano implementation
# meaning that a shallow filter is as accurate as a deep filter
# up to numerical precision errors
print(nllt,nlln,NLL)
ylim(max(ylim()[0],-100),5) 

## Compare to Numpy implementation

In [ ]:
from dstep import filter_moments_dstep_surrogate

tic()
allLRnd,allLVnd,allM1nd,allM2nd,nllnd = filter_moments_dstep_surrogate(D,stim,Y_train,mrt,vrt,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    prior       = (iniM1,iniM2))
toc()

subplot(311)
stderrplot(allLRref,allLVref,color=BLACK,lw=0.5)
stderrplot(allLRnd,allLVnd,color=RUST,lw=0.5,filled=0)
niceaxis()
print(nllnd,NLL)

# Compute gradients?

In [ ]:

theano_shallow_gradient = Tfun(inp = [Xst,TY_,MR_,VR_,par], 
          out = [Tcast(theano.gradient.jacobian(Tcast(_NLL[-1]),Tcast(par)))])

# Check gradients

In [ ]:
large   = sqrt(np.finfo('float32').max)

def objective(p):
    allLR,allLV,M1,M2,NLL = scan_moments_parallel_theano_surrogate(Bh_train,Y,mrt,vrt,p)
    if not isfinite(NLL):
        NLL = large
    return NLL

def gradient(p):
    g = theano_shallow_gradient(Bh_train,Y,mrt,vrt,p)[0]
    return g

g1 = numeric_grad(objective,p,1e-4)
g2 = gradient(p)
print(v2str(g1))
print(v2str(g2))

# Optimize?

In [ ]:
result = p.copy()

In [ ]:
verbose = 1

def og(p):
    o = objective(p)
    g = gradient(p)
    return o,g

try:
    #result = minimize_retry(og,result,jac=True,verbose=verbose,simplex_only=False,options={'eps':1e-3})
    #result = minimize_retry(objective,result,jac=False,verbose=verbose,simplex_only=False,options={'eps':1e-3})
    result = minimize_retry(objective,result,jac=False,verbose=verbose,simplex_only=True)
    print("Finished optimization")
except KeyboardInterrupt:
    print('Optimization paused')
    
print('x=','['+','.join([np.float128(x).astype(str) for x in result])+']')
print("Total absolute change from GLM fit is",sum(abs(result-p0)))


    [-4.47485402448,-6.72860060925,-10.6006414101,16.3551035615,-17.4139653139,19.8456123891,-8.87941524903,-0.53968146647,0.40470884356,0.0081113061503,0.147167177181,0.730789289648,-0.232357798704,0.143245700568,-0.325816468342,0.304508845892,-0.195850447753]